In [5]:
import scanpy as sc

dr = sc.read_h5ad('withDR.h5ad')


In [ ]:
# Filter for C3control cells and create new AnnData object
dr_c3 = dr[dr.obs['sample'] == 'C3control'].copy()
dr_c3

In [7]:
import numpy as np
import scanpy as sc

gene_pass_counts = np.sum(dr_c3.X >= 2, axis=0)
min_cells = int(0.1 * dr_c3.n_obs)
if isinstance(gene_pass_counts, np.matrix):
    gene_pass_counts = np.asarray(gene_pass_counts).squeeze()
gene_filter = gene_pass_counts >= min_cells
dr_c3 = dr_c3[:, gene_filter].copy()


In [ ]:
points_df = dr_c3.uns['points_df']
genes_to_filter = ['ADGRL2', 'BATF3', 'BBS10', 'BLOC1S3', 'CEBPD', 'CENPO', 'CHAF1B', 'DEPDC7', 'EIF4EBP2', 'ENPP4', 'GATA2', 'GOLPH3L', 'LAMTOR3', 'NOL6', 'PLPP5', 'QPRT', 'RARS1', 'SESN3', 'SLC35C1', 'STEAP2', 'TPRA1', 'TTI1', 'UBP1', 'VPS13D', 'VSIG10', 'VWA8', 'XAB2', 'ZNF335']
filtered_points_df = points_df[~points_df['gene'].isin(genes_to_filter)]
filtered_points_df

In [ ]:
filtered_points_df_control = filtered_points_df[filtered_points_df['condition'] == 'Control']
filtered_points_df_control

In [11]:
filtered_points_df_control = filtered_points_df_control[filtered_points_df_control['gene'].isin(dr_c3.var.index)]


In [ ]:
import spatialdata as sd
import json

Control = sd.read_zarr('cellline0101_C3control_2d_dr.zarr')



In [14]:
import pandas as pd
points_df_Control = pd.DataFrame(filtered_points_df_control)
nucleusShapeControl = Control.shapes['nucleus_boundaries']['geometry']
nucleusShapeControl = nucleusShapeControl.astype(object)
cellShapeControl = Control.shapes['cell_boundaries']['geometry']
cellShapeControl = cellShapeControl.astype(object)

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

def plot_cells_in_region_with_genes(xmin, xmax, ymin, ymax, gene_list, ax):
    for cell_id in range(len(cellShapeControl)):
        cell_shape = cellShapeControl.iloc[cell_id]
        if not cell_shape.bounds[0] > xmax and not cell_shape.bounds[2] < xmin and \
           not cell_shape.bounds[1] > ymax and not cell_shape.bounds[3] < ymin:

            nucleus_shape = nucleusShapeControl.iloc[cell_id]

            # Plot cell and nucleus boundaries
            x, y = cell_shape.exterior.xy
            ax.plot(x, y, 'k-', linewidth=1)

            # Plot interior contours
            for contour in cell_shape.interiors:
                x, y = contour.xy
                ax.plot(x, y, 'gray', linewidth=5)

            # Plot nucleus with light blue fill
            x, y = nucleus_shape.exterior.xy
            ax.plot(x, y, '#969696', linewidth=1)
            ax.fill(x, y, color='#969696', alpha=0.5)

            # Filter points for this cell
            cell_points = points_df_Control[(points_df_Control['cell'] == cell_id) & 
                                            (points_df_Control['nuclei'] == 0)]

            for gene in gene_list:
                rb_points = cell_points[cell_points['feature_name'] == f"{gene}_rbRNA"]
                nt_points = cell_points[cell_points['feature_name'] == f"{gene}_ntRNA"]

                if not rb_points.empty:
                    ax.scatter(rb_points['x'], rb_points['y'], c='red', s=20, marker='o', label=f'{gene}_rbRNA', alpha=1)

                if not nt_points.empty:
                    ax.scatter(nt_points['x'], nt_points['y'], c='blue', s=20, marker='o', label=f'{gene}_ntRNA', alpha=1)

            # Draw distance ratio contours
            ratios = [0.33, 0.66]
            styles = ['--', '--']
            colors = ['gray', 'gray']

            nucleus_boundary = nucleus_shape.boundary
            cell_boundary = cell_shape.boundary

            theta = np.linspace(0, 2*np.pi, 100)
            for ratio, style, color in zip(ratios, styles, colors):
                contour_points = []
                for t in theta:
                    r_min = 0
                    r_max = max(cell_shape.bounds[2] - cell_shape.bounds[0], 
                                cell_shape.bounds[3] - cell_shape.bounds[1])
                    
                    while r_max - r_min > 0.1:
                        r = (r_min + r_max) / 2
                        x = nucleus_boundary.centroid.x + r * np.cos(t)
                        y = nucleus_boundary.centroid.y + r * np.sin(t)
                        point = Point(x, y)

                        if cell_shape.contains(point) and not nucleus_shape.contains(point):
                            dn = point.distance(nucleus_boundary)
                            dc = point.distance(cell_boundary)
                            current_ratio = dn / (dn + dc)

                            if abs(current_ratio - ratio) < 0.01:
                                contour_points.append((x, y))
                                break
                            elif current_ratio < ratio:
                                r_min = r
                            else:
                                r_max = r
                        else:
                            if nucleus_shape.contains(point):
                                r_min = r
                            else:
                                r_max = r
                
                if contour_points:
                    contour_points = np.array(contour_points)
                    ax.plot(contour_points[:,0], contour_points[:,1], style, color=color, linewidth=1)

    # Add legend
    handles = [
        plt.Line2D([], [], color='k', linestyle='-', label='Cell boundary'),
        plt.Line2D([], [], color='#969696', linestyle='-', label='Nucleus boundary', 
                  markerfacecolor='#969696', fillstyle='full'),
        plt.Line2D([], [], color='gray', linestyle='--', label='Contours'),
        plt.Line2D([], [], color='blue', marker='o', linestyle='None', label='ntRNA'),
        plt.Line2D([], [], color='red', marker='o', linestyle='None', label='rbRNA')
    ]
    ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5))

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.axis('on')



In [18]:
import pandas as pd
plt.rcParams['svg.fonttype'] = 'none'
genes_df = pd.read_csv('3bin_lower_genes0_1.csv')
gene_list = genes_df['gene'].tolist()

import os
output_dir = "DRdraw"
os.makedirs(output_dir, exist_ok=True)

x_start = 5050
x_end = 6250
y_start = 7100
y_end = 8000

fig, ax = plt.subplots(figsize=(12, 10))
plot_cells_in_region_with_genes(xmin=x_start, xmax=x_end, ymin=y_start, ymax=y_end, gene_list=gene_list, ax=ax)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f'genes136_0_1_{x_start}_{y_start}.svg'), format='svg', bbox_inches='tight')
plt.close(fig)